# Week 4 Day 5 -- Selecting and Combining tools
Starting other programs from Python

## Barcode - from tab-delimited to fasta
Goal: to identify Mystery Mosquito X  
A small piece of mtDNA (part of the CO1 gene) can be 
used to identify the species by comparing to a global
database of mosquito DNA 'barcodes'.

Problem statement and tool scripts by Hendrik-Jan Megens  
Python code by Hendrik-Jan Megens and Mark R. Kramer    

## Preparations
We will use functions `check_call()` and `check_output()` from module `subprocess` instead of `os.popen()` as used in the book.  
The Python documentation discourages use of `os.popen()` and in fact that function does not work properly when an underlying call fails.  

In [1]:
# Import modules/functions we are using
from subprocess import check_call, check_output

Unfortunately, `check_output()` gives an error message in some situations when there the output is empty. Therefore, we create our own `check_output_ext()` that produces the output anyway.  
The code in `check_output_ext` is admittedly more complex than we are teaching in this course. 

In [2]:
import subprocess

def check_output_ext (command, shell=False):
    try:
        output = subprocess.check_output (command, shell=shell)
    except subprocess.CalledProcessError as e:
        output = e.output
    return output

## First explorations

In [3]:
# Define filenames for later reference
barcodes_filename = "bold_Culicidae.tsv"  # primary input file
fasta_filename = 'mosquitos_barcodes.fa'  # intermediate output file
mystery_filename_pattern = 'mystery_*.fa' # input files to be classified

In [4]:
# Open primary input file
infile = open(barcodes_filename, 'r')
# Oops, the file is not there yet...

IOError: [Errno 2] No such file or directory: 'bold_Culicidae.tsv'

Normally, you would download the files manually, or use a shell script for this.  
But now we're doing Python, let's issue the commands from within Python.  
Before doing the download, try some simpler commands first:

In [5]:
# use check_call() to run a command as if entered at the shell, hence shell=True
command = 'ls -l'
check_call(command, shell=True)

0

Outside of a notebook, this would show the directory listing in the console.  
Now it goes to the console... ...of Jupyter. Check the Terminal window where you started Jupyter.

Instead of `check_call()` we can use `check_output()` to obtain the data in a string:

In [6]:
# use check_output() to run command and obtain its output, always using shell=True
command = 'ls -l'
output = check_output(command, shell=True)
print output

total 6144
-rwxr-xr-x 1 lin029 domain users   634 Sep 27 12:10 mystery_mosquito_x1.fa
-rwxr-xr-x 1 lin029 domain users   591 Sep 27 12:10 mystery_mosquito_x2.fa
-rwxr-xr-x 1 lin029 domain users   121 Sep 29 11:38 proteinX.fasta
-rwxr-xr-x 1 lin029 domain users   131 Sep 29 12:37 READ_ME.txt
-rwxr-xr-x 1 lin029 domain users 25917 Sep 29 14:08 W4D5part1.ipynb
-rwxr-xr-x 1 lin029 domain users 12966 Sep 29 12:34 W4D5part2.ipynb



Let's try a more specific `ls`

In [7]:
# use check_output() to run more specific command
command = 'ls -l bold_Culicidae.tsv'
print command
# wait a minute, we had a symbolic name for that filename
command = 'ls -l %s' % barcodes_filename
print command
output = check_output(command, shell=True)
print output

ls -l bold_Culicidae.tsv
ls -l bold_Culicidae.tsv


CalledProcessError: Command 'ls -l bold_Culicidae.tsv' returned non-zero exit status 2

This is exactly the situation for which we need our `check_output_ext()`:

In [8]:
# use check_output_ext() to run more specific command
command = 'ls -l bold_Culicidae.tsv'
print command
# wait a minute, we had a symbolic name for that filename
command = 'ls -l %s' % barcodes_filename
print command
output = check_output_ext(command, shell=True)
print output

ls -l bold_Culicidae.tsv
ls -l bold_Culicidae.tsv



In [9]:
# print lines before and after the output
print '---start of output---'
print output + '-----end of output---'
print 'if file is absent, output will be empty'

---start of output---
-----end of output---
if file is absent, output will be empty


Before continuing, edit and run the preceeding box to avoid getting an empty line.

If there is a barcode file, use `rm` to remove it:

In [10]:
command = 'rm %s' % barcodes_filename
check_call(command, shell=True)

CalledProcessError: Command 'rm bold_Culicidae.tsv' returned non-zero exit status 1

Value 0 means that the underlying command completed successfully.  
Now we download the file using `check_call()` or `check_output()` in Python.

In [11]:
# let's also introduce a symbolic name for the server
server_url = 'http://www.bioinformatics.nl/courses/BIF-50806/Examples/'
command = 'wget %s%s' % (server_url, barcodes_filename)
print command
# either use check_output() ... it succeeds but gives no output
output = check_output(command, shell=True)
print output
# or use check_call() ... it succeeds and gives value 0
check_call(command, shell=True)
#if you keep both, the file is downloaded twice; the second one overwriting the first one

wget http://www.bioinformatics.nl/courses/BIF-50806/Examples/bold_Culicidae.tsv



0

Rerun the appropriate box(es) above to check that the barcodes file is present now.

## Now the real work begins

In [ ]:
# Open primary input file
infile = open(barcodes_filename, 'r')
# Like we tried before; this time it should succeed

Side remark: When using Python 3, some data in a file like this might cause problems.  
Then you have to make explicit that the file is *not* supposed to be Unicode:  
`infile = open(barcodes_filename, 'r', encoding='Latin-1') # Python 3`

In [ ]:
# Close the file again
infile.close()

In order to avoid problems with files being partially read (or not opened, or not closed, we will keep open/process/close inside one cell from now on.  
Sometimes that means that we are closing a file after partially reading it. In later cells we will start over again.

In [ ]:
infile = open(barcodes_filename, 'r')
# Read header line and split into column names
header_line = infile.readline()
header_line = header_line.rstrip()
print header_line
colnames = header_line.split('\t') # tab-delimited
print colnames
# Close the file, even if not fully processed yet
infile.close()

In [ ]:
# Print column names with their positions (index) in the list
for i in range(len(colnames)):
    print i, colnames[i]

We could find column numbers for fields that we want to use by inspecting the list above;
something like:  
`i_species = 20`  
`i_seq_is = 39`  
`i_seq = 42`  
However, we can let Python take care of this:

In [ ]:
i_species = colnames.index('species_name')
i_seq_id = colnames.index('sequenceID')
i_nuc = colnames.index('nucleotides')
# Add one or more lines to show the values of these variables

We are going to read each line from the barcodes file and write two corresponding lines to the fasta file.  
The general structure, including opening and closing both files, is like this:

In [ ]:
infile = open(barcodes_filename, 'r')
fasta_file = open(fasta_filename,'w')

# Process header line
header_line = infile.readline()
# further details above

# Process rest of file line by line
for line in infile:
    # process line and write corresponding output to fasta_file
    # details developed below
    fasta_file.write('') # dummy, will be replaced

fasta_file.close()
infile.close()

Before we actually do that in a loop, let's first walk through the mechanism.  
We create a variable with the contents of a typical data line.  
This code is a substitute for the loop "`for line in infile:`" which gives a line like this every time through the loop.

In [ ]:
line = ( # parentheses to be able to use multiple lines
    # consecutive literal strings are seen as one long string; no + needed in this case
    'GBDCU1576-14\t'              # field 0
    'HG793655\t'                  # field 1
    '5189666\t'                   # field 2
    ' \t'                         # field 3 empty
    'HG793655\t'                  # field 4
    'Mined from GenBank, NCBI\t'  # field 5
    'BOLD:ACF2166\t'              # field 6
    '20\tArthropoda\t'            # fields 7 and 8
    '82\tInsecta\t'               # fields 9 and 10
    '127\tDiptera\t'              # fields 11 and 12
    '1730\tCulicidae\t'           # fields 13 and 14
    '2142\tCulicinae\t'           # fields 15 and 16
    '6441\tCulex\t'               # fields 17 and 18
    '80365\t'                     # field 19
    'Culex torrentium\t'          # field 20: species name
    ' \t \t \t \t \t \t \t \t \t' # fields 21-29 empty
    'isolate: SAW_1551\t'         # field 30
    ' \t \t \t \t \t \t \t \t'    # fields 31-38 empty
    '6836465\t'                   # field 39: sequenceID
    'COI-5P\t'                    # field 40
    'HG793655\t'                  # field 41
    'AACATTATA----CAACATTTA\t\n'  # field 42: nucleotides (shortened)
    )
line # shows the line as if it were defined on one line, including explicit \t etc.

In [ ]:
# Split the line into its elements
line = line.rstrip()
elements = line.split('\t')
# let's look at some key fields
print '#elements:', len(elements)
print elements[0]
print elements[i_species]
print elements[i_seq_id]
print elements[i_nuc]

In [ ]:
# Check if there are enough elements; a line might contain fewer elements
if len(elements) >= len(colnames):
    print "we can continue"
else:
    print "line doesn't have enough elements"
# below we will not print, but instead skip lines that are too short

In [ ]:
# Get fields of interest
species = elements[i_species]
seq_id = elements[i_seq_id]
seq = elements[i_nuc]
# Add one or more lines to show the values of these variables

In [ ]:
# Check if species name is defined
species = species.strip()
if len(species) > 0:
    print "we can continue again"
else:
    print "species name is empty"
# below we will not print, but instead skip lines that contain an empty species name

In [ ]:
# Spaces in names might cause trouble, so replace them by underscores
species = species.replace(' ', '_')
# The sequence may contain dashes because sometimes harvested from alignments
# where missing values or alignment gaps can be inserted with '-'.
# We want to remove these because that might collide with making blastable db
newseq = seq.replace('-', '')
# We make a unique identifier by concatening species name with seqid
newname = '%s|%s' % (species, seq_id)

print species
print newseq
print newname

We will write all values `newname` and `newseq` to a fasta file, in the same loop as reading all lines of the input file.  
Here, we open the fasta file, write two lines to it, and then close the file again.  
In the final version opening and closing should go outside of the loop, before and after the loop, respectively.

In [ ]:
# Before the loop:
fasta_file = open(fasta_filename,'w')
# Ultimately within the loop:
# Write first name, then sequence to fasta file
fasta_file.write('>'+newname+'\n')
fasta_file.write(newseq+'\n')
# After the loop:
fasta_file.close()

Let's use `check_output()` to see what's in our fasta file now.

In [ ]:
# Show content of faste file
command = 'cat %s' % fasta_filename
output = check_output(command, shell=True)
print '---start of output---\n' + output + '-----end of output---'

Putting all these pieces together, we get

In [ ]:
infile = open(barcodes_filename, 'r')
fasta_file = open(fasta_filename,'w')

# Process header line
# Read header line and split into column names
header_line = infile.readline()
header_line = header_line.rstrip()
colnames = header_line.split('\t') # tab-delimited
i_species = colnames.index('species_name')
i_seq_id = colnames.index('sequenceID')
i_nuc = colnames.index('nucleotides')

# Process rest of file line by line
for line in infile:
    # Split the line into its elements
    line = line.rstrip()
    elements = line.split('\t')
    
    # Check if there are enough elements; a line might contain fewer elements
    if len(elements) >= len(colnames):
        
        # Get fields of interest
        species = elements[i_species]
        seq_id = elements[i_seq_id]
        seq = elements[i_nuc]

        # Check if species name is defined
        species = species.strip()
        if len(species) > 0:
            
            # Spaces in names might cause trouble, so replace them by underscores
            species = species.replace(' ', '_')
            # The sequence may contain dashes because sometimes harvested from alignments
            # where missing values or alignment gaps can be inserted with '-'.
            # We want to remove these because that might collide with making blastable db
            newseq = seq.replace('-', '')
            # We make a unique identifier by concatening species name with seqid
            newname = '%s|%s' % (species, seq_id)

            # Write first name, then sequence to fasta file
            fasta_file.write('>'+newname+'\n')
            fasta_file.write(newseq+'\n')

# After the loop
fasta_file.close()
infile.close()

## Processing the mystery mosquito data
When working in the shell, we could now type:

`ls -l mystery*.fa`  
`makeblastdb -in mosquitos_barcode.fa -out mosquitos_barcode -dbtype nucl -parse_seqids`  
`blastn -query mystery_mosquito_x1.fa -db mosquitos_barcode.fa -outfmt 6`  
`blastn -query mystery_mosquito_x2.fa -db mosquitos_barcode.fa -outfmt 6`

But let's just stay with Python for a little longer:

In [ ]:
# Check if mystery files are present
command = 'ls -l %s' % mystery_filename_pattern
output = check_output_ext(command, shell=True)
print '---start of output---\n' + output + '-----end of output---'

If files are missing, download them from Blackboard.

In [ ]:
# Get list of mystery filenames; use ls without -l for getting just names
command = 'ls %s' % mystery_filename_pattern
output = check_output_ext(command, shell=True)
mystery_filenames = output.split('\n')
mystery_filenames = mystery_filenames[:-1] # remove last (empty) entry
print mystery_filenames

In [ ]:
# Make a blastable db of the fasta file, and read in some of the reporting messages 
# that are provided by the 'makeblastdb' program
command = 'makeblastdb -in %s -out %s -dbtype nucl -parse_seqids'
fasta_filename_to_dot = fasta_filename.split('.')[0]
command = command % (fasta_filename, fasta_filename_to_dot)
print command

output = check_output(command, shell=True)
print len(output)
print(output[:200]) # truncate if it is too long

In [ ]:
m_filename = mystery_filenames[0]

# Here things get exciting: we Blast on the blastable db using the sequence of Mystery Mosquito X
# Note that results are collected in the variable 'results' as a string of characters! 
# If you want to loop through the results, you will have to create a list from that string
# You can do that by splitting on a newline character. Note: we are removing the last newline of the full
# character string. If we didn't, there would be an empty last item in the list after the split

command = 'blastn -query %s -db %s -outfmt 6'
command = command % (m_filename, fasta_filename_to_dot)
print command

results = check_output(command, shell=True)
result_lines = results.split('\n')

# We are only interested in the top 5 results:
# You can change this to whatever you want
# You can even make an option for it in the arguments list [challenge].
for result in result_lines[:5]:
    # each item in the list is one line of Blast output (outfmt 6) 
    # Since each line is a string, we need to explicitly split on tabs!
    resultlist = result.split('\t')

    # collect contents of the fields of interest
    mysterymosquito_id = resultlist[0]
    match = resultlist[1]
    percent_match = float(resultlist[2])
    length_of_match = int(resultlist[3])

    # report back the match
    out_line = 'Match for Mystery Mosquito %s: %s with %.2f%% identity for %dbp'
    out_line = out_line % (mysterymosquito_id, match, percent_match, length_of_match)
    print out_line


In [ ]:
# Repeat Blast for all m_filesname entries from mystery_filenames
for m_filename in mystery_filenames:
    # Blast on the blastable db using the sequence of Mystery Mosquito X
    command = 'blastn -query %s -db %s -outfmt 6'
    command = command % (m_filename, fasta_filename_to_dot)
    print command

    results = check_output(command, shell=True)
    result_lines = results.split('\n')

    # We are only interested in the top 5 results:
    # You can change this to whatever you want
    # You can even make an option for it in the arguments list [challenge].
    for result in result_lines[:5]:
        # each item in the list is one line of Blast output (outfmt 6) 
        # Since each line is a string, we need to explicitly split on tabs!
        resultlist = result.split('\t')

        # collect contents of the fields of interest
        mysterymosquito_id = resultlist[0]
        match = resultlist[1]
        percent_match = float(resultlist[2])
        length_of_match = int(resultlist[3])

        # report back the match
        out_line = 'Match for Mystery Mosquito %s: %s with %.2f%% identity for %dbp'
        out_line = out_line % (mysterymosquito_id, match, percent_match, length_of_match)
        print out_line
